# Reduced Rank Model

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from models.reduced_rank_model import ReducedRankModel

In [2]:
DATA_PATH = 'raw_data/full_data/'
TOTAL_SESSIONS = 3

files = os.listdir(DATA_PATH)
X_files = [f for f in files if f.endswith('.npy') and f.startswith('X')]
X_files.sort()
Y_files = [f for f in files if f.endswith('.npy') and f.startswith('Y')]
Y_files.sort()

X_data = []
for f in X_files[:TOTAL_SESSIONS]:
    X_data.append(np.load(DATA_PATH + f))

Y_data = []
for f in Y_files[:TOTAL_SESSIONS]:
    Y_data.append(np.load(DATA_PATH + f))

# Training

In [ ]:
# ReducedRankModel